In [39]:
cd C:\Users\ARITRAS ROY\Desktop\Aritras\FaceRecognitionSVD

C:\Users\ARITRAS ROY\Desktop\Aritras\FaceRecognitionSVD


#### Author: Kanan Vyas
### Program: Face recognition using Singular Vector Decomposition

In [150]:
# Importing libraries
import numpy as np
import os
import matplotlib.pyplot as plt
from matplotlib.image import imread
import cv2

In [151]:
# Giving path for training images
Path2 = './dataimage/'
files2 = os.listdir(Path2)
images = []

In [152]:
#Readling all the images and storing into an array
for name in files2:
    temp = cv2.imread(Path2+name)
    temp = cv2.cvtColor(temp,cv2.COLOR_BGR2GRAY)
    temp = cv2.resize(temp,(100,100), interpolation = cv2.INTER_AREA)
    images.append(temp.flatten())
images = np.array(images)
images.shape

(31, 10000)

In [153]:
images

array([[203, 203, 203, ..., 176, 176, 176],
       [203, 203, 203, ..., 177, 177, 177],
       [192, 202, 203, ..., 136, 141, 130],
       ...,
       [203, 203, 201, ..., 177, 177, 177],
       [203, 203, 203, ..., 177, 177, 177],
       [203, 203, 203, ..., 176, 176, 161]], dtype=uint8)

In [154]:
# Substractng mean from all images for normalization    
mu = np.mean(images)
images = images-mu
images = images.T
print(images.shape)

(10000, 31)


In [155]:
# Performing SVD
u,s,v = np.linalg.svd(images, full_matrices=False)
print(u.shape, s.shape , v.shape)

(10000, 31) (31,) (31, 31)


In [156]:
# Reading test image as an input and converting into 100x100 
test = np.array(cv2.imread('./test_images/subject11.rightlight.jpg'))
test = cv2.cvtColor(test,cv2.COLOR_BGR2GRAY)
test = cv2.resize(test,(100,100), interpolation = cv2.INTER_AREA)
img = test.reshape(1, -1)
img.shape

(1, 10000)

In [157]:
# Substracting mean
img = img-mu  # normalization of test image
img = img.T
print(img[:][50])

[71.41584839]


In [158]:
img.shape

(10000, 1)

In [159]:
print(img[50])

[71.41584839]


In [162]:
test_x = np.empty(shape = (u.shape[0], u.shape[1]),  dtype=np.int8)
print(test_x.shape)

(10000, 31)


In [163]:
img[:,0]

array([ -4.58415161,   8.41584839,  -6.58415161, ..., -96.58415161,
       -87.58415161, -87.58415161])

In [164]:
u.shape

(10000, 31)

In [165]:
col = 30
test_x[:,col] = img[:,0] * u[:,col]
test_x

array([[ 80,   1,  23, ..., 111, 111,   0],
       [122, 122, 124, ..., 112, 113,   0],
       [113, 118, 118, ..., 125, 125,   0],
       ...,
       [ 14,  15,  15, ...,  32,  32,   0],
       [ 35,  35,  35, ...,  23,  23,   0],
       [ 24,  24,  24, ...,  38,  57,   0]], dtype=int8)

In [166]:
# Dot product of test image and U matrix
for col in range(u.shape[1]):    
    test_x[:,col] = img[:,0] * u[:,col]  # dotting the test image with all columns of U

dot_test = np.array(test_x, dtype='int8').flatten()
dot_test.shape

(310000,)

In [167]:
# Dot product of all the images and U matrix
dot_train = np.empty(shape = (u.shape[0]*u.shape[1], u.shape[1]),  dtype=np.int8)
temp = np.empty(shape = (u.shape[0], u.shape[1]),  dtype=np.int8)

for i in range(images.shape[1]):
    for c in range(u.shape[1]):    
        temp[:,c] = images[:,i] * u[:,c]   # dotting each training image with all columns of U
    tempF = np.array(temp, dtype='int8').flatten()
    dot_train[:,i] = tempF[:]
    
dot_train.shape

(310000, 31)

In [ ]:
# dot_train 

In [168]:
# Substracting two dot products (difference between test data and training data)
sub = np.empty(shape = (u.shape[0]*u.shape[1], u.shape[1]))

for col in range(u.shape[1]):
    sub[:,col] = dot_train[:,col] - dot_test[:]
    
sub.shape

(310000, 31)

In [169]:
a = sub[1]
a

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [170]:
# Finding norm of all the columns
answer = np.empty(shape=(u.shape[1],))

for c in range(sub.shape[1]):    
    answer[c] = np.linalg.norm(sub[:,c])

print(answer)

[508.79563677 512.82550639 520.69376797 458.71886815 520.23071805
 506.93786602 516.26059311 462.52351292 457.38277187 557.2934595
 554.92882426 593.34728448 614.43632705 465.18598431 494.18114088
 531.75370238 537.03351851 561.83271532 561.83271532 434.25338226
 497.16295115 337.99852071 315.30461462 509.15812868 582.45858222
 475.12735135 532.96998039 594.80585068 563.73043203 529.46387979
 496.61755909]


In [171]:
# Sorting answer array and retrieving first element which will be minimum from all
temp_ans = np.empty(shape=(u.shape[1],))
temp=np.copy(answer)

temp.sort()
check = temp[0]
print(check)

315.30461461894276


In [172]:
answer.shape[0]

31

In [173]:
index=0

for i in range(answer.shape[0]):
    if check == answer[i]:
        index=i
        break
i

22

In [174]:
# Checking for corresponding image for minimum norm
folder_tr = '/dataimage/'
i = 0
print(index)
for filename in os.listdir(os.getcwd()+"/"+folder_tr):
    if index == i:
        print("The predicted face is: ",filename)
        break
    else:
        i=i+1

22
The predicted face is:  subject11.normal.jpg
